# Welcome to Colab!

In [ ]:
#----------------------------------------------
# Part I: IoT Device Data Encryption Simulation
#----------------------------------------------
import json
import random
from Crypto.Cipher import AES
import base64
import os

# Generate AES-128 bit symmetric key (16 bytes)
key = os.urandom(16)

# Generate initialization vector for CBC mode — must also be 16 bytes
iv  = os.urandom(16)


def IoT_Device():
    # 1) Generate sensor readings (simulating temperature + humidity values)
    temperature = round(random.uniform(15, 40), 2)
    humidity    = round(random.uniform(20, 80), 2)

    data = {"temp": temperature, "humidity": humidity}

    # 2) Convert JSON → Bytes for encryption
    plaintext = json.dumps(data).encode()

    # 3) Add PKCS#7 padding to make length a multiple of 16 bytes (AES block size)
    pad = 16 - len(plaintext) % 16
    plaintext += bytes([pad]) * pad

    # 4) Encrypt using AES CBC mode
    cipher = AES.new(key, AES.MODE_CBC, iv)
    encrypted = cipher.encrypt(plaintext)

    # 5) Encode to Base64 to be safe for network transmission
    payload = base64.b64encode(encrypted).decode()

    return payload  # This is what the IoT device will send to the server


def Server(payload):
    # 1) Decode Base64 back to encrypted bytes
    encrypted_bytes = base64.b64decode(payload)

    # 2) Decrypt AES CBC using same key + IV
    cipher = AES.new(key, AES.MODE_CBC, iv)
    decrypted = cipher.decrypt(encrypted_bytes)

    # 3) Remove padding to restore original plaintext
    pad = decrypted[-1]
    decrypted = decrypted[:-pad]

    # 4) Convert bytes → JSON dictionary
    data = json.loads(decrypted.decode())

    return data


# IoT device generates + encrypts data
payload = IoT_Device()
print("Payload sent:", payload[:50] + "...")  # Only printing part of payload

# Server receives + decrypts data
received_data = Server(payload)
print("Received Data:", received_data)


Payload sent: D1c27oBm09u6rmSv4Ri8WX+O4Pi95C+UtpX70EH28qIH/YJSpE...
Received Data: {'temp': 21.85, 'humidity': 38.96}


In [ ]:
!pip install pycryptodome


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 25.7 MB/s eta 0:00:00
